This code-along analyses data from a survey about the growth of Finnish companies. The data reports the perceptions of top managers on growth, innovativeness, and the ability for renewal.

### Where is the data from?

- [Suominen & Pihlajamaa, 2022](https://www.sciencedirect.com/science/article/pii/S2352340922005261)
- [The dataset](https://zenodo.org/records/5820394#.Y5OKl-zMK3I)

### What will I learn today?

- How to summarize and visualize questions with a numeric response using a histogram.
- How to determine whether there is a difference between two groups of numeric responses using a Mann-Whitney U test.
- How to summarize and visualize questions with a categorical response using a bar plot.

In [1]:
import plotly.express as px
from scipy.stats import mannwhitneyu

In [3]:
SELECT *
    FROM 'data/raw/survey_data.csv'

SyntaxError: invalid syntax (3362231377.py, line 1)